In [1]:
import io 
import re 
import string 
import tensorflow as tf
import tqdm

from tensorflow.keras import Model 
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
%load_ext tensorboard

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


<h1>POSITIVE SAMPLING <h2> skip-gram

In [5]:
vocab, index = {}, 0
vocab['<pad>'] = 0
for token in tokens:
    vocab[token] = index
    index +=1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 5, 'wide': 1, 'road': 2, 'shimmered': 3, 'in': 4, 'hot': 6, 'sun': 7}


In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 5: 'the', 1: 'wide', 2: 'road', 3: 'shimmered', 4: 'in', 6: 'hot', 7: 'sun'}


In [7]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[5, 1, 2, 3, 4, 5, 6, 7]


In [8]:
window_size = 2
positive_skip_gram, _=tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size = vocab_size,
    window_size=window_size,
    negative_samples=0
)
print(len(positive_skip_gram))

26


In [9]:
for target, context in positive_skip_gram[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(2, 1): (road, wide)
(3, 1): (shimmered, wide)
(1, 2): (wide, road)
(5, 2): (the, road)
(4, 2): (in, road)


<h1> NEGATIVE SAMPLING <h2> skip-gram

In [10]:
target_word, context_word = positive_skip_gram[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word,dtype="int64"), (1,1))
negative_smapling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes = context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    seed=SEED,
    name="negative_sampling"
)
print(negative_smapling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_smapling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['road', 'wide', 'in', 'shimmered']


In [11]:
#Construct one training example
